# <span style="color:green">3. Sélection de variables via Stepwise</span>

C'est une étape importante qui demande à la fois une validation statistique et métier .

Le principe est de garder que les variables qui ont un impact statistique prouvé.

Pour cela, on adopte la méthodologie "Stepwise" adaptée au Scoring
Puisque nous devons travailler au niveau modalité, nous allons:

- Binariser (One-Hot Encoding) toutes les variables discrétisées (en prenant soin pour chaque variable de supprimer une modalité de référence).

- Appliquer une sélection Stepwise (Backward Elimination) : On lance un modèle Logit avec TOUTES les modalités. On regarde la P-value de Wald pour chaque coefficient. Si la pire P-value > 0.05, on retire cette modalité spécifique. On relance le modèle et on répète jusqu'à ce que tout soit significatif.

- Vérification de la contrainte "Groupe" : On vérifie à la fin si, pour une variable donnée (ex: person_income), toutes les modalités restantes sont cohérentes.

#### <span style="color:orange">Librairies</span>

In [1]:
import os
import sys
import numpy as np
import pandas as pd


import itertools
from sklearn.metrics import confusion_matrix
import plotly.graph_objects as go
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)

sys.path.append(os.path.abspath("..")) 
from src.utils import plot_default_rates

#### <span style="color:orange">Données</span>

In [2]:
df = pd.read_csv("../datasets/processed/credit_risk_dataset_discretized_final.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32572 entries, 0 to 32571
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   person_age                  32572 non-null  int64
 1   person_income               32572 non-null  int64
 2   person_home_ownership       32572 non-null  int64
 3   person_emp_length           32572 non-null  int64
 4   loan_intent                 32572 non-null  int64
 5   loan_grade                  32572 non-null  int64
 6   loan_amnt                   32572 non-null  int64
 7   loan_int_rate               32572 non-null  int64
 8   loan_status                 32572 non-null  int64
 9   loan_percent_income         32572 non-null  int64
 10  cb_person_default_on_file   32572 non-null  int64
 11  cb_person_cred_hist_length  32572 non-null  int64
dtypes: int64(12)
memory usage: 3.0 MB


In [4]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,0,0,0,1,4,1,0,3,0,0,0,0
1,0,0,1,0,1,2,0,3,1,3,0,0
2,0,4,2,0,1,2,3,5,1,3,0,0
3,0,2,2,1,1,2,3,4,1,3,1,0
4,0,0,0,0,5,0,0,1,1,1,0,0


#### <span style="color:orange">Visualisations rapides</span> 

In [5]:
plot_default_rates(df, df.columns.drop('loan_status'), n_cols=4)

- Certaines variables ne respectent pas le principe de monotonie (sens unique selon les bins) et/ou ne sont pas assez discrétisées (trop de modalités)
- Il est important qu'elles respectent cela pour la selection des variables.


In [6]:
# Regroupement de certaines variables
df['person_age'] = df['person_age'].replace({3: 2})
df['person_income'] = df['person_income'].replace({3: 2, 4: 3, 5: 3})
df['loan_intent'] = df['loan_intent'].replace({2: 1, 3: 2, 4: 2, 5: 3})
df['cb_person_cred_hist_length']= df['cb_person_cred_hist_length'].replace({2: 1})

- `person_age` : On fait ces regroupements pour 2 raisons. D'abord, le respect de la monotonie. Les taux de défaut observés pour les modalités 2 et 3 ne respectaient pas strictement la relation monotone attendue entre l’âge et le risque de défaut. Leur regroupement permet de lisser cette irrégularité et de rétablir une progression cohérente. Ensuite, les effectifs faibles par classe. En effet, Les modalités 2 et 3 sont faiblement représentées dans l’échantillon. Les fusionner permet d’obtenir une modalité plus stable statistiquement, avec des effectifs suffisants pour produire des taux de défaut plus robustes.
- `person_income` : Nous regroupons les modalités 2 et 3 ensemble, puis les modalités 4 et 5 ensemble, afin de réduire le nombre de tranches et d’obtenir une échelle plus lisible tout en conservant l’ordre naturel de la variable. Plus le revenu est élevé, plus le risque de défaut diminue (toutes choses égales par ailleurs). 
- `loan_intent` : On regroupe 1 (MEDICAL) et 2 (HOMEIMPROVEMENT) puis 3 (PERSONAL) et 4 (EDUCATION).
A priori, on ne devrait pas le faire car ces classes sont de natures différentes et on perd de l’information utile. Il y a aussi un risque de non-stabilité temporelle et tout simplement une mauvaise justification règlementaire. On fait cela pour les raisons suivantes. D’abord, la proximité des taux de défaut consécutifs observés. Regrouper des catégories aux taux de défaut similaires pourrait augmenter la stabilité du modèle tout en respectant la monotonie .Ensuite, cela permet une augmentation de la proportion d’individus par classes. On réduit ainsi la variance et on a moins de classes pour le modèle. Enfin, on simplifie le modèle tout en le rendant robuste aux potentiels risques de surajustement. 

In [7]:
df.to_csv('../datasets/processed/credit_risk_dataset_discretized_updated.csv')

In [8]:
plot_default_rates(df, df.columns.drop('loan_status'), n_cols=4)

#### <span style="color:orange">Préselection de variables</span> 

Après observation de la matrice de corrélation, les variables `loan_int_rate`, `loan_grade` et `person_age` seront exclues du modèle par la suite.

- `loan_int_rate` et `loan_grade`: D’un  point de vue métier, ces varaibles sont à exclure car elles sont purement contractuelles, ce qui n’est pas à utiliser pour un modèle de PD. D’un point de vue statistique, ces variables sont  fortement correlées ce qui peut entrainer de la multicolinéarité.
- `person_age` : cette variable est très faiblement corrélée à la cible et ne respecte pas le principe de monotomie.

De plus, certaines variables pourraient entrer en conflit entre elles car assez correlées entre elles. Par exemple, `loan_percent_income`, `person_income` et `loan_amnt` qui sont liées.

In [9]:
# Variable expliquée
y = df["loan_status"]

# Variables explicatives possibles
X = ['person_age', 'person_income', 'person_home_ownership',
     'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
     'loan_int_rate', 'loan_percent_income',
     'cb_person_default_on_file', 'cb_person_cred_hist_length']

# Variables à exclure car non-pertinentes
to_drop_pertinence = ["loan_grade", 'loan_int_rate']  # Variables niveau contrat
X = [var for var in X if var not in to_drop_pertinence] 

# # Variables à exclure car pas monotones ou mauvaise correlation 
# to_drop_no_monotone = ['loan_amnt']
# X = [var for var in X if var not in to_drop_no_monotone] 

# One-hot-encoding de toutes les variables explicatives
encoder = OneHotEncoder(sparse_output=False)  
encoded = encoder.fit_transform(df[X]) 
X_encoded = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(X))

In [10]:
# On supprime les modalités pour lesquelles le taux de défaut observé est le plus faible 
# Elles seront nos modalités de reférence
to_drop = [
    "person_age_2",
    "person_income_3",
    "person_home_ownership_0",
    "person_emp_length_2",
    "loan_intent_3",
    "loan_percent_income_0",
    "cb_person_default_on_file_0", 
    "cb_person_cred_hist_length_1",
    "loan_amnt_0"
    ]

# On garde seulement les colonnes existantes dans X_encoded
to_drop_present = [c for c in to_drop if c in X_encoded.columns]
to_drop_missing = [c for c in to_drop if c not in X_encoded.columns]

if to_drop_missing:
    print("Attention, ces colonnes demandées pour drop n'existent pas et seront ignorées :", to_drop_missing)
if to_drop_present:
    print("Suppression des colonnes:", to_drop_present)

X_encoded = X_encoded.drop(columns=to_drop_present)

Suppression des colonnes: ['person_age_2', 'person_income_3', 'person_home_ownership_0', 'person_emp_length_2', 'loan_intent_3', 'loan_percent_income_0', 'cb_person_default_on_file_0', 'cb_person_cred_hist_length_1', 'loan_amnt_0']


#### <span style="color:orange">Train/Validation Split</span>

In [11]:
# Découpage train / validation pour les modèles itératifs
X_train, X_val, y_train, y_val = train_test_split(
    X_encoded, y,
    test_size=0.25,        # 25% validation
    random_state=42,       # reproductibilité
    stratify=y             # Même proportion de défauts
)
print(X_train.shape, X_val.shape)

(24429, 20) (8143, 20)


In [12]:
X_comb = df[X]
# Découpage train / validation pour les modèles combinatoires
X_train_comb, X_val_comb, y_train_comb, y_val_comb = train_test_split(
    X_comb, y,
    test_size=0.25,        # 25% validation
    random_state=42,       # reproductibilité
    stratify=y             # Même proportion de défauts
)
print(X_train_comb.shape, X_val_comb.shape)

(24429, 9) (8143, 9)


#### <span style="color:orange">Modèle naif de base</span>

In [13]:
import pandas as pd

# ---------------------
# Préparation X / y
# ---------------------
X_train_sm = sm.add_constant(X_train)  # intercept
y_train_sm = y_train.copy()
# ---------------------
# Fit du modèle logit
# ---------------------
model = sm.Logit(y_train_sm, X_train_sm)
result = model.fit(disp=False)
# ---------------------
# Métriques
# ---------------------
# BIC
bic = result.bic
# McFadden R2
r2_mcfadden = 1 - result.llf / result.llnull

print(f"BIC : {bic:.2f}")
print(f"R2 : {r2_mcfadden:.2f}")

BIC : 18591.40
R2 : 0.28


In [14]:
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return '_'

In [15]:
# Récupération des coefficients du modèle
coeffs = result.params
pvals = result.pvalues

rows = []

for var in X:
    # retrouver toutes les colonnes du groupe
    group_cols = [c for c in X_train.columns if c.startswith(var + "_")]

    # variable numérique
    if len(group_cols) == 0:
        coef = coeffs.get(var, np.nan)
        pval = pvals.get(var, np.nan)
        rows.append([var, var + " — (numérique)", coef, pval, significance_stars(pval)])
        continue

    all_modalities = sorted(df[var].dropna().unique())

    # parcourir les modalités dans l'ordre naturel
    for m in all_modalities:
        col_name = f"{var}_{m}"
        if col_name in group_cols:
            coef = coeffs.get(col_name, np.nan)
            pval = pvals.get(col_name, np.nan)
            rows.append([var, col_name, coef, pval, significance_stars(pval)])
        else:
            # modalité de référence
            full_label = f"{col_name} (Référence)"
            rows.append([var, full_label, 0.0, "_", "_"])

# Tableau final
results_table = pd.DataFrame(rows, columns=[
    "Variable", "Modalité", "Coefficient", "p-value", "Significativité"
])

print("\n================ COEFFICIENTS DU MODÈLE DE REGRESSION LOGISTIQUE NAÏF ===============")
display(results_table.reset_index(drop=True))


================ COEFFICIENTS DU MODÈLE DE REGRESSION LOGISTIQUE NAÏF ===============


,Variable,Modalité,Coefficient,p-value,Significativité
0,person_age,person_age_0,0.108081,0.250353,_
1,person_age,person_age_1,0.002746,0.969922,_
2,person_age,person_age_2 (Référence),0.000000,_,_
3,person_income,person_income_0,1.530543,0.0,***
4,person_income,person_income_1,1.082109,0.0,***
5,person_income,person_income_2,0.329761,0.0,***
6,person_income,person_income_3 (Référence),0.000000,_,_
7,person_home_ownership,person_home_ownership_0 (Référence),0.000000,_,_
8,person_home_ownership,person_home_ownership_1,1.649028,0.0,***
9,person_home_ownership,person_home_ownership_2,2.554029,0.0,***


In [16]:
# Export Excel
results_table.reset_index(drop=True).to_excel(
    "../result/3_selection_variables/Logit_naif.xlsx",
    index=False,
)

Problème : Certains coefficients sortent négatifs et/ou ne sont pas significatifs. Cependant, ça reste une bonne baseline pour notre benchmark.

#### <span style="color:orange">Logit avec Stepwise selection</span>

In [17]:
def stepwise_bic_forward_backward(X, y, cat_vars, verbose=True, alpha=0.05, maxiter=100):
    """
    Stepwise (forward + backward) par groupe de modalités (variable complète) avec critère BIC.
    Ensuite, on retire les variables qui ont au moins une modalité non significative (p > alpha).

    """

    # utilitaires
    def make_exog_from_vars(var_list):
        """Construit la DataFrame exogène alignée sur y.index pour la liste de variables var_list."""
        if len(var_list) == 0:
            # DataFrame intercept-only aligné sur y.index
            X_tmp = pd.DataFrame({"const": np.ones(len(y))}, index=y.index)
            X_tmp = sm.add_constant(X_tmp, has_constant='add')
            # remove duplicate const if any
            X_tmp = X_tmp.loc[:, ~X_tmp.columns.duplicated()]
            return X_tmp
        else:
            cols = []
            for v in var_list:
                cols.extend([c for c in X.columns if c.startswith(v + "_")])
            # reindex pour forcer l'alignement avec y
            X_sel = X[cols].reindex(index=y.index)
            X_tmp = sm.add_constant(X_sel, has_constant='add')
            X_tmp = X_tmp.loc[:, ~X_tmp.columns.duplicated()]
            return X_tmp

    def fit_model_from_vars(var_list):
        """Retourne (result, bic) pour la liste de variables var_list (groupes complets)."""
        X_tmp = make_exog_from_vars(var_list)
        model = sm.Logit(y.reindex(X_tmp.index), X_tmp)
        res = model.fit(disp=False, maxiter=maxiter)
        return res, res.bic

    # 1) initial: modèle intercept seul
    selected = []
    result_null, current_bic = fit_model_from_vars(selected)
    if verbose:
        print(f"BIC intercept seul : {current_bic:.2f}")

    improving = True
    iteration = 0
    while improving:
        iteration += 1
        improving = False

        # ---- Forward step : essayer d'ajouter chaque variable non sélectionnée ----
        best_candidate = None
        best_bic_forward = current_bic

        candidates = [v for v in cat_vars if v not in selected]
        for cand in candidates:
            try:
                _, bic = fit_model_from_vars(selected + [cand])
            except Exception:
                # si le fit échoue pour ce candidat, on l'ignore
                continue
            if bic < best_bic_forward:
                best_bic_forward = bic
                best_candidate = cand

        # si on a trouvé un ajout qui améliore le BIC -> l'ajouter
        if best_candidate is not None:
            selected.append(best_candidate)
            current_bic = best_bic_forward
            improving = True
            if verbose:
                print(f"\nItération {iteration} - Forward : + ajout de '{best_candidate}' -> BIC = {current_bic:.2f}")

            # ---- Backward step : après ajout, tenter de retirer des variables ----
            removed = True
            while removed and len(selected) > 0:
                removed = False
                best_bic_backward = current_bic
                worst_var = None

                for cand_remove in selected:
                    try:
                        _, bic = fit_model_from_vars([v for v in selected if v != cand_remove])
                    except Exception:
                        continue
                    if bic < best_bic_backward:
                        best_bic_backward = bic
                        worst_var = cand_remove

                if worst_var is not None:
                    selected.remove(worst_var)
                    current_bic = best_bic_backward
                    removed = True
                    if verbose:
                        print(f"   Backward : - retrait de '{worst_var}' -> BIC = {current_bic:.2f}")

        else:
            # Aucun candidat apportant une amélioration du BIC
            if verbose:
                print("\nAucun ajout possible (forward) qui améliore le BIC. Fin des étapes stepwise BIC.\n")
            break

    if verbose:
        print(f"\nSélection finale par BIC (avant filtrage p-values) : {selected}")
        print(f"BIC final : {current_bic:.2f}\n")

    # 2) Filtrage : conserver seulement les variables dont TOUTES les modalités encodées sont significatives
    def fit_result_from_vars(var_list):
        """Return fitted result for var_list (raises on failure)."""
        X_tmp = make_exog_from_vars(var_list)
        return sm.Logit(y.reindex(X_tmp.index), X_tmp).fit(disp=False, maxiter=maxiter)

    # si aucune variable sélectionnée, on retourne l'intercept
    if len(selected) == 0:
        result_final = result_null
        final_selected = []
    else:
        final_selected = selected.copy()
        while True:
            result = fit_result_from_vars(final_selected)
            pvals = result.pvalues.to_dict()
            # pour chaque variable, vérifier ses modalités encodées
            vars_to_remove = []
            for v in final_selected:
                cols_v = [c for c in X.columns if c.startswith(v + "_")]
                # on vérifie uniquement les colonnes présentes dans le modèle (alignées)
                cols_in_model = [c for c in cols_v if c in pvals]
                non_sig = [c for c in cols_in_model if (pd.isna(pvals[c]) or pvals[c] > alpha)]
                if len(non_sig) > 0:
                    vars_to_remove.append((v, non_sig))

            if len(vars_to_remove) == 0:
                # toutes les modalités restantes sont significatives
                result_final = result
                break
            else:
                # on retire toutes les variables qui ont au moins une modalité non-significative
                removed_vars = [v for v, _ in vars_to_remove]
                for rv in removed_vars:
                    final_selected.remove(rv)
                if verbose:
                    print("Filtrage p-values : retrait des variables (au moins une modalité non-significative) ->", removed_vars)
                if len(final_selected) == 0:
                    # plus de variables -> garder l'intercept
                    result_final = result_null
                    break
                # sinon on refitte dans la boucle et re-vérifie

    if verbose:
        print(f"\nSélection finale après filtrage p-values (alpha={alpha}) : {final_selected}")
        if result_final is not None:
            print(f" • BIC modèle final : {getattr(result_final, 'bic', np.nan):.2f}")
            print(f" • Pseudo R² : {getattr(result_final, 'prsquared', np.nan):.4f}")
            print(f" • Nb paramètres : {len(getattr(result_final, 'params', []))}\n")

    # 3) Construire le tableau final modalité / coef / p-value / significativité
    rows = []
    # build map of coef/pval if we have a fitted result with parameters
    if result_final is not None and hasattr(result_final, "params"):
        coef_map = result_final.params.to_dict()
        pval_map = result_final.pvalues.to_dict()
    else:
        coef_map = {}
        pval_map = {}

    for var in final_selected:
        modalities = sorted(df[var].dropna().unique())
        for m in modalities:
            col_name = f"{var}_{m}"
            # --- IMPORTANT: afficher le vrai nom de la modalité (str(m)) dans la colonne "Modalité"
            if col_name in coef_map:
                coef = coef_map[col_name]
                pval = pval_map.get(col_name, np.nan)
                modalite_label = f"{var}_{m}"
            else:
                # modalité de référence (absente de l'encodage)
                coef = 0.0
                pval = "_"   # <-- remplacer NaN par "_" demandé
                modalite_label = f"{var}_{m} (Référence)"

            rows.append({
                "Variable": var,
                "Modalité": modalite_label,
                "Coefficient": coef,
                "p-value": pval
            })

    table_stepwise = pd.DataFrame(rows)

    # fonction d'étoiles (robuste si p n'est pas numérique)
    def signif_stars(p):
        try:
            pnum = float(p)
        except Exception:
            return "_"
        if pd.isna(pnum):
            return "_"
        if pnum <= 0.001:
            return "***"
        if pnum <= 0.01:
            return "**"
        if pnum <= 0.05:
            return "*"
        if pnum <= 0.1:
            return "."
        return "_"

    if not table_stepwise.empty:
        table_stepwise["Significativité"] = table_stepwise["p-value"].apply(signif_stars)

    return final_selected, result_final, table_stepwise

In [18]:
print("=== Stepwise forward+backward (critère BIC) puis filtrage p-values ===\n")

selected_vars, result_stepwise, table_stepwise = stepwise_bic_forward_backward(
    X=X_train,         # DataFrame encodé des variables explicatives
    y=y_train,         # Série cible binaire
    cat_vars=X,        # Liste des variables à tester (catégorielles)
    verbose=True,      # Affiche les étapes
    alpha=0.05,        # Seuil de significativité pour filtrer les modalités
    maxiter=100        # Max itérations pour l'optimiseur
)

print("\n Statistiques du modèle final :")
if result_stepwise is not None:
    print(f" • BIC : {getattr(result_stepwise, 'bic', np.nan):.2f}")
    print(f" • Pseudo R² : {getattr(result_stepwise, 'prsquared', np.nan):.4f}")
    print(f" • Nombre de paramètres : {len(getattr(result_stepwise, 'params', []))}")
else:
    print("Aucun modèle final (seul l'intercept ou erreur de fit).")

print("\n================ COEFFICIENTS DU MODÈLE DE REGRESSION LOGISTIQUE AVEC STEPWISE ===============")
display(table_stepwise.reset_index(drop=True))  # tableau final sans index


=== Stepwise forward+backward (critère BIC) puis filtrage p-values ===

BIC intercept seul : 25640.96

Itération 1 - Forward : + ajout de 'loan_percent_income' -> BIC = 21706.00

Itération 2 - Forward : + ajout de 'person_home_ownership' -> BIC = 20216.33

Itération 3 - Forward : + ajout de 'cb_person_default_on_file' -> BIC = 19528.61

Itération 4 - Forward : + ajout de 'person_income' -> BIC = 18930.15

Itération 5 - Forward : + ajout de 'loan_intent' -> BIC = 18561.98

Itération 6 - Forward : + ajout de 'loan_amnt' -> BIC = 18554.04

Aucun ajout possible (forward) qui améliore le BIC. Fin des étapes stepwise BIC.


Sélection finale par BIC (avant filtrage p-values) : ['loan_percent_income', 'person_home_ownership', 'cb_person_default_on_file', 'person_income', 'loan_intent', 'loan_amnt']
BIC final : 18554.04

Filtrage p-values : retrait des variables (au moins une modalité non-significative) -> ['loan_amnt']

Sélection finale après filtrage p-values (alpha=0.05) : ['loan_percent_inc

,Variable,Modalité,Coefficient,p-value,Significativité
0,loan_percent_income,loan_percent_income_0 (Référence),0.000000,_,_
1,loan_percent_income,loan_percent_income_1,0.327552,0.0,***
2,loan_percent_income,loan_percent_income_2,0.618926,0.0,***
3,loan_percent_income,loan_percent_income_3,2.946857,0.0,***
4,person_home_ownership,person_home_ownership_0 (Référence),0.000000,_,_
5,person_home_ownership,person_home_ownership_1,1.689757,0.0,***
6,person_home_ownership,person_home_ownership_2,2.595670,0.0,***
7,cb_person_default_on_file,cb_person_default_on_file_0 (Référence),0.000000,_,_
8,cb_person_default_on_file,cb_person_default_on_file_1,1.175669,0.0,***
9,person_income,person_income_0,1.395905,0.0,***


In [19]:
# Export Excel
table_stepwise.reset_index(drop=True).to_excel(
    "../result/3_selection_variables/Logit_Stepwise.xlsx",
    index=False
)

#### <span style="color:orange">Modèles combinatoires</span> 

Ici, on essaie différentes modélisations avec une méthode combinatoire. On utilise directement les variables discrétisées et on teste toutes les combinaisons de k variables parmi les 9 disponibles. On regarde la significativité et on classe les modèles par BIC. L'avantage est qu'on explore toutes les combinaisons possibles de variables.

##### <span style="color:orange">Modèles à 2 variables </span> 

In [36]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def combi_logit_bic(X, y):
    # Réindexage X sur y pour garantir l'alignement 
    X = X.reindex(index=y.index)

    cols = X.columns.tolist()
    results = []

    for var1, var2 in itertools.combinations(cols, 2):
        # Matrice des explicatives
        X_temp = X[[var1, var2]].copy()
        # On aligne d'abord X et y , ensuite on dropna pour le fit
        mask = X_temp.notna().all(axis=1) & y.notna()
        X_fit = X_temp.loc[mask]
        y_fit = y.loc[mask]

        X_sm = sm.add_constant(X_fit)

        # try:
        model = sm.Logit(y_fit, X_sm)    # Regression logistique
        res = model.fit(disp=False, maxiter=200)
        bic = res.bic

        # Coeffs et p-values / NaN si absence
        coef_v1 = res.params.get(var1, np.nan)
        coef_v2 = res.params.get(var2, np.nan)
        pval_v1 = res.pvalues.get(var1, np.nan)
        pval_v2 = res.pvalues.get(var2, np.nan)

        results.append({
            "var1": var1,
            "var2": var2,
            "coef_var1": coef_v1,
            "coef_var2": coef_v2,
            "pval_var1": pval_v1,
            "pval_var2": pval_v2,
            "BIC": bic
        })

    df_results = pd.DataFrame(results)

    # Tri par BIC décroissant
    df_results = df_results.sort_values(by="BIC", ascending=True).reset_index(drop=True)

    return df_results

# Exécution
df_combi_results = combi_logit_bic(X_train_comb, y)
df_combi_results.head()

,var1,var2,coef_var1,coef_var2,pval_var1,pval_var2,BIC
0,person_home_ownership,loan_percent_income,1.100573,0.902782,1.238539e-259,0.000000e+00,20634.282273
1,person_income,loan_percent_income,-0.482471,0.773274,7.279171e-164,0.000000e+00,21303.204818
2,loan_percent_income,cb_person_default_on_file,0.900274,1.094322,0.000000e+00,2.739996e-162,21328.698850
3,loan_intent,loan_percent_income,-0.365675,0.902589,8.384449e-91,0.000000e+00,21623.484184
4,person_emp_length,loan_percent_income,-0.314451,0.878815,5.288641e-30,0.000000e+00,21907.444969


##### <span style="color:orange">Modèles à n variables </span>

In [41]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from math import comb

def combi_k_logit_bic(X, y, k):
    """
    Tester toutes les combinaisons de k variables dans X pour une régression logit sur y.
    Garde la même logique que les fonctions précédentes (réindexage, check too_few_obs, tri par BIC, arrondi à 3 décimales).
    
    Parameters
    ----------
    X : pandas.DataFrame
        DataFrame des variables explicatives (une colonne = une variable)
    y : pandas.Series
        Variable cible binaire
    k : int
        Taille des combinaisons à tester (ex. 2,3,4,...)
    
    Returns
    -------
    df_results : pandas.DataFrame
        DataFrame des résultats (colonnes var1..vark, coef_var1..coef_vark, pval_var1..pval_vark, BIC, éventuellement error)
    """
    # Réindexage X sur y pour garantir l'alignement 
    X = X.reindex(index=y.index)

    cols = X.columns.tolist()
    n_cols = len(cols)
    n_models = comb(n_cols, k) if n_cols >= k and k > 0 else 0
    print(f"Nombre de modèles possibles : {n_models} classés par BIC")

    results = []

    for combo in itertools.combinations(cols, k):
        # Matrice des explicatives (k variables)
        X_temp = X[list(combo)].copy()

        # Alignement + dropna pour le fit
        mask = X_temp.notna().all(axis=1) & y.notna()
        X_fit = X_temp.loc[mask]
        y_fit = y.loc[mask]

        # Préparer les clés dynamiques
        row = {}
        for i, var in enumerate(combo, start=1):
            row[f"var{i}"] = var

        # Si trop peu d'observations, on saute (enregistrant l'erreur comme précédemment)
        if len(y_fit) < 10:
            for i in range(1, k+1):
                row[f"coef_var{i}"] = np.nan
                row[f"pval_var{i}"] = np.nan
            row["BIC"] = np.nan
            row["error"] = "too_few_obs"
            results.append(row)
            continue

        X_sm = sm.add_constant(X_fit)

        model = sm.Logit(y_fit, X_sm)
        res = model.fit(disp=False, maxiter=200)
        bic = res.bic

        # Récupérer coeffs et p-values pour chaque variable du combo
        for i, var in enumerate(combo, start=1):
            row[f"coef_var{i}"] = res.params.get(var, np.nan)
            row[f"pval_var{i}"] = res.pvalues.get(var, np.nan)

        row["BIC"] = bic

        results.append(row)

    df_results = pd.DataFrame(results)

    # Tri par BIC ascendant (les plus petits BIC en premier)
    df_results = df_results.sort_values(by="BIC", ascending=True, na_position='last').reset_index(drop=True)

    # Limiter tous les résultats numériques à 3 décimales
    df_results = df_results.round(3)

    return df_results


In [42]:
combi5 = combi_k_logit_bic(X_train_comb, y, 5)
combi5.head(5)

Nombre de modèles possibles : 126 classés par BIC


,var1,var2,var3,var4,var5,coef_var1,pval_var1,coef_var2,pval_var2,coef_var3,pval_var3,coef_var4,pval_var4,coef_var5,pval_var5,BIC
0,person_income,person_home_ownership,loan_intent,loan_percent_income,cb_person_default_on_file,-0.436,0.000,0.976,0.0,-0.359,0.000,0.853,0.0,1.122,0.000,19145.183
1,person_income,person_home_ownership,loan_amnt,loan_percent_income,cb_person_default_on_file,-0.530,0.000,1.010,0.0,0.179,0.000,0.734,0.0,1.086,0.000,19437.387
2,person_income,person_home_ownership,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,-0.429,0.000,0.994,0.0,0.833,0.000,1.103,0.0,0.037,0.316,19496.832
3,person_income,person_home_ownership,person_emp_length,loan_percent_income,cb_person_default_on_file,-0.426,0.000,0.990,0.0,-0.021,0.493,0.833,0.0,1.103,0.000,19497.367
4,person_age,person_income,person_home_ownership,loan_percent_income,cb_person_default_on_file,0.014,0.633,-0.428,0.0,0.993,0.000,0.833,0.0,1.103,0.000,19497.610


In [44]:
# Export Excel
combi5.head(5).reset_index(drop=True).to_excel(
    "../result/3_selection_variables/Logits_5_variables.xlsx",
    index=False,
)

#### <span style="color:orange">Modèle Final </span>

On garde le logit Stepwise car :
- Sélection de variables pertinente : On ajoute progressivement les variables qui améliorent le BIC et retire celles qui le détériorent. Cela permet de rester parcimonieux : moins de variables, moins de bruit, meilleur équilibre biais/variance.De plus, on garantit que toutes les modalités retenues sont statistiquement significatives (p < 0.05 par défaut).

- Interprétabilité : La régression logistique produit des coefficients directement interprétables en termes de probabilités ou d’odds ratios. Chaque coefficient indique l’impact relatif d’une modalité sur le risque de défaut. De plus, cela est conforme aux exigences réglementaires (explicabilité, auditabilité, rapport à un régulateur)

- Robustesse et contrôle du sur-apprentissage : Le critère BIC pénalise le nombre de paramètres. Cette pénalisation réduit le risque de sur-apprentissage, contrairement à une régression logistique complète avec toutes les variables.

##### <span style="color:orange">Evaluation du Modèle Final </span>

In [20]:
# Colonnes réellement utilisées par le modèle final
model_cols = result_stepwise.model.exog_names
has_const = "const" in model_cols # On sépare la constante et variables explicatives
vars_no_const = [c for c in model_cols if c != "const"]
# Sélection des variables explicatives de X_val
X_val_model = X_val[vars_no_const].copy()
X_val_model = sm.add_constant(X_val_model, has_constant="add") # On rajoute la constante
# Sécurisation de l'ordre des colonnes
X_val_model = X_val_model[model_cols]

In [21]:
# Probabilités de défaut sur l'échantillon de validation
y_val_pred_proba = result_stepwise.predict(X_val_model)
# Predictions binaires avec seuil à 0.5
y_val_pred = (y_val_pred_proba >= 0.5).astype(int)

In [22]:
cm = confusion_matrix(y_val, y_val_pred)

fig = go.Figure(
    data=go.Heatmap(
        z=cm,
        x=["Prédit 0", "Prédit 1"],
        y=["Réel 0", "Réel 1"],
        text=cm,
        texttemplate="%{text}",
        colorscale="Blues",
        showscale=True
    )
)
fig.update_layout(
    title="Matrice de confusion – Échantillon de validation",
    xaxis_title="Classe prédite",
    yaxis_title="Classe réelle",
    width=500,
    height=500
)
fig.show()

In [23]:
print("\n=== Performances sur l'échantillon de validation ===")
print(f"Accuracy        : {accuracy_score(y_val, y_val_pred):.3f}")
print(f"Precision       : {precision_score(y_val, y_val_pred):.3f}")
print(f"Recall (Sens.)  : {recall_score(y_val, y_val_pred):.3f}")
print(f"F1-score        : {f1_score(y_val, y_val_pred):.3f}")
print(f"AUC ROC         : {roc_auc_score(y_val, y_val_pred_proba):.3f}")


=== Performances sur l'échantillon de validation ===
Accuracy        : 0.845
Precision       : 0.749
Recall (Sens.)  : 0.437
F1-score        : 0.552
AUC ROC         : 0.814


In [24]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc

# ROC
fpr, tpr, _ = roc_curve(y_val, y_val_pred_proba)
roc_auc = auc(fpr, tpr)

fig = go.Figure()
# Courbe ROC
fig.add_trace(
    go.Scatter(
        x=fpr,
        y=tpr,
        mode="lines",
        name=f"Logit Stepwise (Validation) – AUC = {roc_auc:.3f}",
        line=dict(width=2)
    )
)
# Diagonale (modèle aléatoire)
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode="lines",
        name="Modèle aléatoire",
        line=dict(dash="dash", color="grey")
    )
)
fig.update_layout(
    title="Courbe ROC – Échantillon de validation",
    xaxis_title="Taux de faux positifs (FPR)",
    yaxis_title="Taux de vrais positifs (TPR)",
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1]),
    width=600,
    height=500,
    legend=dict(x=0.6, y=0.05)
)
fig.show()

#### <span style="color:orange">Prochaine étape : Grille de score</span>